In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.preprocessing import RobustScaler

# Load dataset
df = pd.read_csv('f1_2019_to_2023_all_drivers_all_data.csv', low_memory=False)

# Convert time columns to seconds
time_columns = ['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time']
for col in time_columns:
    df[col] = pd.to_timedelta(df[col]).dt.total_seconds()

# Convert binary columns to integer type
df['Rainfall'] = df['Rainfall'].astype(int)
df['FreshTyre'] = df['FreshTyre'].astype(int)
df['IsAccurate'] = df['IsAccurate'].astype(int)


# Categorize weather condition based on centroid values of Kmeans clustering
def categorize_weather(row):
    if row['Rainfall'] > 0:
        return 'Rainy'
    elif row['AirTemp'] > 28.43213126:
        return 'high'
    elif row['AirTemp'] > 21.31279265:
        return 'medium'
    elif row['AirTemp'] > 12.84901403:
        return 'low'
    else:
        return 'very_low'
df['Weather_Category'] = df.apply(categorize_weather, axis=1)
df = pd.get_dummies(df, columns=['Weather_Category'])


# Create Track temperature category based on the result of Kmeans clustering 
df['TrackTemp_Cat'] = pd.cut(df['TrackTemp'], bins=[0, 18.96764999, 27.87457484, 35.04425766, 41.75142602, 50.51006013, 53.02449646], labels=['VERY_LOW', 'Low', 'Medium', 'Warm', 'High','VERY_High'])
df = pd.get_dummies(df, columns=['TrackTemp_Cat'])


# One-hot encoding
df = pd.get_dummies(df, columns=['Driver', 'Compound', 'Team','TrackStatus','Circuit'])
# Drop irrelevant columns
columns_to_drop = ['Time', 'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
                   'PitOutTime', 'PitInTime', 'LapStartDate', 'Deleted', 'DeletedReason', 'FastF1Generated',
                   'IsPersonalBest', 'Sector3Time','LapStartTime','Sector2Time','Sector1Time']
df.drop(columns=columns_to_drop, inplace=True)


## Separate Rainy / dry days ##
# 1. Separate LapTime as dry or wet(rainy) condition ( since lapTime of rainy day would be recognized as outliers)
# 2. Remove Outliers for dry condition LapTime
# 3. Build Combined LapTime df (Outliers for dry days are deleted)

# Flag for rainy conditions
df['IsRainy'] = df['Rainfall'].apply(lambda x: 1 if x > 0 else 0)

# Separate dataframes for dry and wet conditions
df_dry = df[df['IsRainy'] == 0]
df_wet = df[df['IsRainy'] == 1]


def remove_outliers(df, column_name, multiplier=1.5):
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    return df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

# Apply standard IQR for dry days
df_dry_filtered = remove_outliers(df_dry, 'LapTime', multiplier=1.5)

# Apply a more lenient IQR for wet days
df_wet_filtered = remove_outliers(df_wet, 'LapTime', multiplier=2.0)


df_combined = pd.concat([df_dry_filtered, df_wet_filtered], ignore_index=True)


# Define features and target
X = df_combined.drop('LapTime', axis=1)
y = df_combined['LapTime']


train_years = [2019,2020,2021,2022]
test_year = 2023
# Split data based on year
X_train = df_combined[df_combined['Year'].isin(train_years)].drop(['LapTime', 'Year'], axis=1)
y_train = df_combined[df_combined['Year'].isin(train_years)]['LapTime']
X_test = df_combined[df_combined['Year'] == test_year].drop(['LapTime', 'Year'], axis=1)
y_test = df_combined[df_combined['Year'] == test_year]['LapTime']




# Drop rows where the target variable is missing in the training set
train_indices = y_train.dropna().index  # Indices of rows where y_train is not NaN
X_train = X_train.loc[train_indices]
y_train = y_train.dropna()  # Drop missing values in y_train

numeric_features = ['Humidity', 'Pressure', 'WindDirection', 'WindSpeed','TrackTemp','AirTemp','SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST']

X_train[numeric_features] = X_train[numeric_features].fillna(method='ffill')

# Forward fill missing values in the test set
X_test[numeric_features] = X_test[numeric_features].fillna(method='ffill')

# Scale the test set using the same scaler fitted on the training set



scaler = RobustScaler()
X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])
X_test[numeric_features] = scaler.transform(X_test[numeric_features])


/var/folders/_5/533f3cq93vq25vl2f3txpkb40000gn/T/ipykernel_1216/2388303341.py:106: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_train[numeric_features] = X_train[numeric_features].fillna(method='ffill')
/var/folders/_5/533f3cq93vq25vl2f3txpkb40000gn/T/ipykernel_1216/2388303341.py:109: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_test[numeric_features] = X_test[numeric_features].fillna(method='ffill')


In [2]:

#1. Linear regression model with cross validation

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error


# Initialize and fit Linear Regression
linear = LinearRegression()
linear.fit(X_train, y_train)

linear.fit(X_train, y_train)
y_pred = linear.predict(X_test)

rmse = mean_squared_error(y_test,y_pred,squared=False)
print(rmse)


# Evaluate with K-Fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(linear, X_train, y_train, cv=kfold, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-scores)  # Convert MSE to RMSE
print("Cross-validated RMSE scores:", rmse_scores)
print("Mean RMSE:", np.mean(rmse_scores))

4.734507551293419
Cross-validated RMSE scores: [3.42618768 3.50579253 3.44386084 3.45867339 3.47477468]
Mean RMSE: 3.4618578236323736


In [3]:

from sklearn.model_selection import RandomizedSearchCV, KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Initialize and fit Random Forest with specified parameters
rf_model = RandomForestRegressor(max_depth=30, min_samples_leaf=1, min_samples_split=10, n_estimators=200, random_state=42,n_jobs=-1)
rf_model.fit(X_train, y_train)

# Predict on test set
rf_predictions = rf_model.predict(X_test)

# Calculate RMSE
rf_rmse = mean_squared_error(y_test, rf_predictions, squared=False)
print(f"Random Forest RMSE: {rf_rmse:.3f}")

# Evaluate with K-Fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf_model, X_train, y_train, cv=kfold, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-scores)  # Convert MSE to RMSE
print("Cross-validated RMSE scores:", rmse_scores)
print("Mean RMSE:", np.mean(rmse_scores))

Random Forest RMSE: 8.052
Cross-validated RMSE scores: [1.84872689 1.85824514 1.87492179 1.86537882 1.76326769]
Mean RMSE: 1.842108066210674


In [4]:
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

##BASE XG BOOST

#Grid value :(n_estimators=700, learning_rate=0.1,random_state=42,n_jobs=-1,max_depth=7)

# Train the model with class weight adjustment
xgb_model = XGBRegressor(random_state=42,reg_alpha=10,  # L1 regularization
                         reg_lambda=1)
xgb_model.fit(X_train, y_train)

# Predictions and evaluation
predictions = xgb_model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
print(f"Baseline RMSE for Combined df: {rmse}")

from sklearn.model_selection import cross_val_score

# Evaluate with K-Fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42,)
scores = cross_val_score(xgb_model, X_train, y_train, cv=kfold, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-scores)  # Convert MSE to RMSE
print("Cross-validated RMSE scores:", rmse_scores)
print("Mean RMSE:", np.mean(rmse_scores))


Baseline RMSE for Combined df: 5.365817002847865
Cross-validated RMSE scores: [1.86925626 1.93032117 1.8025015  1.89152527 1.79397305]
Mean RMSE: 1.8575154497159034


In [ ]:
#4. XG boost with Random Search

import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import make_scorer, mean_squared_error
kf = KFold(n_splits=5, shuffle=True, random_state=42)
xgb_model = xgb.XGBRegressor(random_state=42)
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800],
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    'max_depth': [3, 5, 7, 9, 11],
    'reg_alpha': [0.1, 1, 10, 100],
    'reg_lambda': [0.1, 1, 10, 100],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0]
}
neg_rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=True)
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, n_iter=100, cv=kf,
                                   scoring={'RMSE': neg_rmse_scorer}, refit='RMSE', random_state=42, verbose=3, n_jobs=-1)
random_search.fit(X, y)
best_model = random_search.best_estimator_
best_params = random_search.best_params_
best_rmse = (-random_search.best_score_) ** 0.5  # Converting MSE to RMSE

print("Best model parameters:", best_params)
print(f"Optimized CV RMSE: {best_rmse:.3f}")


Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [ ]:
"""import shap
import numpy as np
import pandas as pd

# Use SHAP to explain feature importance
explainer = shap.Explainer(xgb_model)
shap_values = explainer(X_test)

# Calculate the mean absolute value for each feature to represent importance
shap_sum = np.abs(shap_values.values).mean(axis=0)
feature_importance_shap = pd.Series(shap_sum, index=X_test.columns)

# Scale the importances so that they sum to 100%
feature_importance_shap_scaled = 100 * feature_importance_shap / feature_importance_shap.sum()

# Print all scaled feature importances
print("Feature importances scaled to 100%:")
for feature, importance in feature_importance_shap_scaled.sort_values(ascending=False).items():
    print(f"{feature}: {importance:.2f}%")

# Define feature groups
weather_keywords = ['Pressure', 'Temp', 'Humidity', 'Rainfall', 'WindSpeed', 'WindDirection', 'Weather_Category', 'TrackTemp_Cat']
weather_features = [col for col in X_test.columns if any(keyword in col for keyword in weather_keywords)]
circuit_features = [col for col in X_test.columns if 'Circuit' in col]

# Summarize weather impacts
weather_importance = sum(importance for feature, importance in feature_importance_shap_scaled.items() if feature in weather_features)
print(f"Total importance of weather features scaled to 100%: {weather_importance:.2f}%")

# Summarize circuits impacts
circuit_importance = sum(importance for feature, importance in feature_importance_shap_scaled.items() if feature in circuit_features)
print(f"Total importance of circuit features scaled to 100%: {circuit_importance:.2f}%")

# Plot feature importance using SHAP
shap.summary_plot(shap_values, X_test, feature_names=X_test.columns)"""

In [ ]:
"""# Assuming xgb_model has already been trained
feature_importances = xgb_model.feature_importances_
sorted_indices = np.argsort(feature_importances)[::-1]

# Creating a list to store feature names and their importances
features_sorted = []
for index in sorted_indices:
    features_sorted.append((X_train.columns[index], feature_importances[index]))

# Now features_sorted contains all the features and their importances sorted by importance
weather_features = ['Pressure', 'AirTemp', 'Humidity', 'Rainfall', 'WindSpeed', 'TrackTemp', 'WindDirection']
circuit_features = [col for col in X_train.columns if 'Circuit' in col]

# Summarize weather impacts
weather_importance = sum(importance for feature, importance in features_sorted if feature in weather_features)
print(f"Total importance of weather features: {weather_importance}")

# Summarize circuits impacts
circuit_importance = sum(importance for feature, importance in features_sorted if feature in circuit_features)
print(f"Total importance of circuit features: {circuit_importance}")"""

In [ ]:
"""import pandas as pd
import matplotlib.pyplot as plt

# Calculate Spearman correlation matrix for df_combined
correlation_matrix = df_combined.corr(method='spearman')

# Focus on 'LapTime' correlations
laptime_correlations = correlation_matrix['LapTime'].drop('LapTime')  # drop self-correlation

# Convert correlations to percentage and sort in descending order
laptime_correlations_percentage = laptime_correlations * 100
laptime_correlations_sorted = laptime_correlations_percentage.sort_values(ascending=False)

# Convert to list and format the output
laptime_correlations_list = [(f"{index}: {value:.2f}%") for index, value in laptime_correlations_sorted.items()]

# Print the formatted list
for item in laptime_correlations_list:
    print(item)

# Optionally, visualize the correlations in descending order
plt.figure(figsize=(10, 8))
laptime_correlations_sorted.plot(kind='bar')
plt.title('Percentage Correlation of LapTime with Other Variables')
plt.ylabel('Correlation Percentage')
plt.xlabel('Variables')
plt.grid(True)
plt.show()"""